# Solving Bellmann equation on a simple example

In this tutorial we will solve the Bellman-equation on a simple grid world problem which can be seen in the image below. Here are the rules:

* The gray areas are fields where the game ends. When we step on them, it is over. 
* The game can start from any field. 
* In each step the agent receives reward -1. 
* On each field the agent can move one step away in the directions indicated by the arrows at left.
* The agent can not leave the table. At the edge a step, which would lead beyond the table, won't have effect.

![gridworld](https://drive.google.com/uc?export=download&id=1zXSXzcT1zH1w9111VxJ1F28B7JF2X33n)

The Bellman-equation as we saw earlier:

\begin{equation}
V^\pi(s) = \sum_{a, s'}{\left( r(s, a, s') + \gamma V^\pi(s') \right)T(s, a, s')\pi(s, a)}
\end{equation}

* $T(s, a, s')$ is the transition matrix
* $\pi(s, a)$ is the policy
* $V^\pi(s')$ is the state-value function
* $r(s, a, s')$ is the reward.

If we have a fixed policy $\pi$, we can calculate the corresponding $V^\pi(s)$ by solving the Bellman-equation. This can be achieved by using the following iteration:

\begin{equation}
V^\pi_{t+1}(s) = \sum_{a, s'}{\left( r(s, a, s') + \gamma V^\pi_t(s') \right) T(s, a, s') \pi(s, a)}
\end{equation}

It can be proved that no metter how the $V^\pi_0$ is chosen the iteration will converge to the same value $V^\pi_\infty$. This process is the **policy evaluation**.

According to $V^\pi_\infty$ we calculate another policy $\pi'$ which can be different from $\pi$. Then we change $\pi$ to $\pi'$ at different $s$ states where the expected return is bigger. This is the **policy improvement**.

The policy can be calculated as follows:

\begin{equation}
\pi(s) = argmax_a\sum_{s'}{\left( r(s, a, s') + \gamma V^*_\infty(s) \right) T(s, a, s')}
\end{equation}

where $V^*_\infty$ is the optimal state-value function.

In [1]:
import numpy as np

In [2]:
# We need a matrix to store the transition probabilities. (transition matrix)

T_sas = np.zeros((16, 16, 4)) # we have 16 states, 4 actions

H = np.array([x for x in range(16)])
T_sas[H, H - (H % 4 != 0), 0] = 1 # left
T_sas[H, H - 4*(H-4 >= 0), 1] = 1 # up
T_sas[H, H + ((H+1) % 4 != 0), 2] = 1 # right
T_sas[H, H + 4*(H+4 < 16), 3] = 1 # down
T_sas[0, :, :] = 0 # in the shaded area each action is pointless
T_sas[0, 0, :] = 1
T_sas[15, :, :] = 0
T_sas[15, 15, :] = 1

In [3]:
# reward
r_sas = -np.ones((16, 16, 4))

r_sas[0, 0, :] = 0
r_sas[15, 15, :] = 0

In [4]:
# Bellmann-equation
def bellman_operator(p_sa, v_s, gamma=0.98):
    rT = np.sum(r_sas * T_sas, 1)
    rTpi = np.sum(rT * p_sa, 1)
    
    pi = p_sa.reshape(16, 1, 4).repeat(16, 1)
    vT = np.sum(T_sas * pi, 2)
    vTpi = np.matmul(vT, v_s) * gamma
    return rTpi + vTpi

In [5]:
# policy improvement
def policy_improvement(v_s, gamma=0.98):
    rT = np.sum(r_sas * T_sas, 1)
    v = v_s.reshape(1, 16, 1).repeat(16, 0).repeat(4, 2)
    pi_idx = np.argmax(rT + gamma * np.sum(T_sas * v, 1), 1)
    pi = np.zeros((16, 4))
    pi[np.array([x for x in range(16)]), pi_idx] = 1
    return pi

In [6]:
# value iteration step
def value_iteration(gamma=0.98):
    rT = np.sum(r_sas * T_sas, 1)
    v = v_s.reshape(1, 16, 1).repeat(16, 0).repeat(4, 2)
    return np.max(rT + gamma * np.sum(T_sas * v, 1), 1)

In [7]:
# policy
def policy(v_s, gamma=0.98):
    rT = np.sum(r_sas * T_sas, 1)
    v = v_s.reshape(1, 16, 1).repeat(16, 0).repeat(4, 2)
    return np.argmax(rT + gamma * np.sum(T_sas * v, 1), 1)

In [8]:
# policy evaluation and policy improvement
gamma = 0.98
v_s = np.zeros(16)
p_sa = np.ones((16, 4)) * 0.25 # random policy
for _ in range(50):
    for _ in range(20): # policy evaluation
        v_s = bellman_operator(p_sa, v_s, gamma)
    p_sa = policy_improvement(v_s, gamma) # policy improvement
pi = policy(v_s, gamma)

In [9]:
pi.reshape((4, 4))

array([[0, 0, 0, 0],
       [1, 0, 0, 3],
       [1, 0, 2, 3],
       [1, 2, 2, 0]], dtype=int64)

In [10]:
v_s.reshape((4, 4)).astype(np.float32)

array([[ 0.    , -1.    , -1.98  , -2.9404],
       [-1.    , -1.98  , -2.9404, -1.98  ],
       [-1.98  , -2.9404, -1.98  , -1.    ],
       [-2.9404, -1.98  , -1.    ,  0.    ]], dtype=float32)

In [11]:
# Value iteration to solve the problem
gamma = 0.9
v_s = np.zeros(16)
p_sa = np.zeros((16, 4)) * 0.25
for _ in range(200):
    v_s = bellman_operator(p_sa, v_s, gamma)
    p_sa = policy_improvement(v_s, gamma)
pi = policy(v_s, gamma)

In [12]:
pi.reshape((4, 4))

array([[0, 0, 0, 0],
       [1, 0, 0, 3],
       [1, 0, 2, 3],
       [1, 2, 2, 0]], dtype=int64)

In [13]:
v_s.reshape((4, 4)).astype(np.float32)

array([[ 0.  , -1.  , -1.9 , -2.71],
       [-1.  , -1.9 , -2.71, -1.9 ],
       [-1.9 , -2.71, -1.9 , -1.  ],
       [-2.71, -1.9 , -1.  ,  0.  ]], dtype=float32)

We can see that the state-values are quite reasonable. They are close to the number of steps required to achieve the closest shaded area. 